In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

import warnings
from collections import Counter
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import PolynomialFeatures
warnings.filterwarnings('ignore')

import pickle
import xgboost as xgb

import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV #KFold 
from sklearn.cross_validation import StratifiedKFold, KFold
from sklearn.metrics import confusion_matrix, mean_squared_error, mean_absolute_error
from sklearn.datasets import load_iris, load_digits, load_boston
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

import time
import pygeohash as pgh
import geopy.distance

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
community_health = pd.read_csv("../DATA/community_health.csv")

In [15]:
service_reqs = pd.read_feather("../DATA/311_service_requests.feather")

In [13]:
mask = service_reqs['complaint_type'].values == 'Dirty Conditions'
service_reqs[mask].description.value_counts()

E3 Dirty Sidewalk                   71716
E15 Illegal Postering               50682
E3A Dirty Area/Alleyway             48184
E1 Improper Disposal                26261
E12 Illegal Dumping Surveillance    25821
E8 Canine Violation                 22919
E5 Loose Rubbish                    10065
E2 Receptacle Violation              9618
E11 Litter Surveillance              7849
E4 18" Law                           5811
E1A Litter Basket / Improper Use     2585
E13 Throw-Out                        1536
E7 Private Carter Spillage            247
E3B Sidewalk Obstruction              222
Name: description, dtype: int64

In [6]:
service_reqs.complaint_type.value_counts()

Water System                     526705
Dirty Conditions                 283516
Sanitation Condition             241938
Rodent                           212545
Food Establishment                66151
Air Quality                       60458
Indoor Air Quality                37548
Food Poisoning                    24652
Hazardous Materials               21966
Asbestos                          16319
Smoking                           15090
Drinking                          10823
Water Quality                      9686
Mold                               2644
Annual Inspection                  2316
Animal                             1425
Drinking Water                      873
Damaged or Dead Tree                449
BBQ Outside Authorized Area         175
Health                              164
Calorie Labeling                    126
Annual/ Cycle Inspection            117
Trans Fat                            28
Animal Waste                         26
Illegal Animal - Sold/Kept           22


In [7]:
service_reqs.description.value_counts()

15 Street Cond/Dump-Out/Drop-Off                               157461
Rat Sighting                                                   109122
Leak (Use Comments) (WA2)                                      102991
Hydrant Running Full (WA4)                                      77069
E3 Dirty Sidewalk                                               71716
12 Dead Animals                                                 69424
Hydrant Running (WC3)                                           67212
Dirty Water (WE)                                                64856
Condition Attracting Rodents                                    54572
E15 Illegal Postering                                           50682
E3A Dirty Area/Alleyway                                         48184
Fire Hydrant Emergency (FHE)                                    46015
Hydrant Leaking (WC1)                                           37018
Mouse Sighting                                                  33843
E1 Improper Disposal

In [20]:
service_reqs.created.min()

Timestamp('2010-01-01 01:00:00+0000', tz='UTC')

In [21]:
service_reqs.created.max()

Timestamp('2018-03-13 02:56:54+0000', tz='UTC')

In [28]:
service_reqs.created.dt.year.value_counts()

2017    211987
2016    211176
2015    203562
2010    185211
2014    172764
2011    172274
2012    166830
2013    163814
2018     48252
Name: created, dtype: int64

In [38]:
nta_indicators = pd.read_excel("../DATA/Appendix_Housing_and_Health_Indicators_by_NTA_072418.xlsx").iloc[2:]
nta_indicators.columns = nta_indicators.iloc[0]

In [41]:
nta_indicators = nta_indicators.iloc[1:]

In [60]:
ntai = nta_indicators.iloc[:188,0:7]


In [61]:
ntas = pd.read_csv("../DATA/demographics_city.csv")[['nta_name', 'nta_code']]
ntas.columns

Index(['nta_name', 'nta_code'], dtype='object')

In [62]:
ntai.columns = ['nta_name', 'borough', 'nta_code', 'crowded_housing', 'ED_visits_per_100', 'medicaid_concentration', 'medicaid_enrollees']

In [68]:
ntai.shape[0] == ntas.shape[0]

True

In [85]:
ntai.to_csv("../DATA/nta_health_response.csv")

In [73]:
demo_c = pd.read_csv("../DATA/clean_demo.csv")
demo_c

,Unnamed: 0,key,nta_name,population,people_per_acre,households,median_income,mean_income
0,1,BX31,Allerton-Pelham Gardens,28903,39.8,9587,28903,28903
1,2,SI01,Annadale-Huguenot-Prince's Bay-Eltingville,27770,8.4,9885,27770,27770
2,3,SI48,Arden Heights,25238,21.8,8960,25238,25238
3,4,QN70,Astoria,78793,87.3,34310,78793,78793
4,5,QN48,Auburndale,19996,25.5,7122,19996,19996
5,6,QN76,Baisley Park,34603,35.7,10730,34603,34603
6,7,BK27,Bath Beach,29931,62.3,11125,29931,29931
7,8,MN25,Battery Park City-Lower Manhattan,39699,82.7,21216,39699,39699
8,9,BK31,Bay Ridge,78607,50.5,34991,78607,78607
9,10,QN46,Bayside-Bayside Hills,43808,23.6,16008,43808,43808


In [74]:
demo_c['household_occupancy'] = demo_c.population / demo_c.households

In [75]:
demo_c

,Unnamed: 0,key,nta_name,population,people_per_acre,households,median_income,mean_income,household_occupancy
0,1,BX31,Allerton-Pelham Gardens,28903,39.8,9587,28903,28903,3.014812
1,2,SI01,Annadale-Huguenot-Prince's Bay-Eltingville,27770,8.4,9885,27770,27770,2.809307
2,3,SI48,Arden Heights,25238,21.8,8960,25238,25238,2.816741
3,4,QN70,Astoria,78793,87.3,34310,78793,78793,2.296502
4,5,QN48,Auburndale,19996,25.5,7122,19996,19996,2.807638
5,6,QN76,Baisley Park,34603,35.7,10730,34603,34603,3.224884
6,7,BK27,Bath Beach,29931,62.3,11125,29931,29931,2.690427
7,8,MN25,Battery Park City-Lower Manhattan,39699,82.7,21216,39699,39699,1.871182
8,9,BK31,Bay Ridge,78607,50.5,34991,78607,78607,2.246492
9,10,QN46,Bayside-Bayside Hills,43808,23.6,16008,43808,43808,2.736632


In [81]:
nta_health_data = pd.read_excel("../DATA/nta-metadata.xlsx", "NTA Data")

In [83]:
nta_health_data['key'] = nta_health_data.NTA_Code

In [89]:
demo_c.merge(nta_health_data, on='key')[["NTA_Code","PreventHospDiab"]].to_csv("../DATA/nta_health_response.csv")

In [99]:
nta_features = demo_c.merge(nta_health_data[["key","TobaccoRetailers", "Daycares", "MedicaidEnroll", "HomeDeliveredMeals", "FoodStampSNAP"]], on='key').iloc[:,1:]       

In [101]:
nta_features.to_csv("../DATA/nta_health_features.csv")

,key,nta_name,population,people_per_acre,households,median_income,mean_income,household_occupancy,TobaccoRetailers,Daycares,MedicaidEnroll,HomeDeliveredMeals,FoodStampSNAP,nta_code
0,BX31,Allerton-Pelham Gardens,28903,39.8,9587,28903,28903,3.014812,8.0,46.0,27.7,12.0,16.6,BX31
1,SI01,Annadale-Huguenot-Prince's Bay-Eltingville,27770,8.4,9885,27770,27770,2.809307,5.5,17.0,11.5,17.0,3.8,SI01
2,SI48,Arden Heights,25238,21.8,8960,25238,25238,2.816741,1.2,8.0,14.1,12.0,4.2,SI48
3,QN70,Astoria,78793,87.3,34310,78793,78793,2.296502,14.9,59.0,27.4,17.0,11.7,QN70
4,QN48,Auburndale,19996,25.5,7122,19996,19996,2.807638,6.2,13.0,26.2,11.0,6.8,QN48
5,QN76,Baisley Park,34603,35.7,10730,34603,34603,3.224884,8.6,81.0,33.8,18.0,22.9,QN76
6,BK27,Bath Beach,29931,62.3,11125,29931,29931,2.690427,10.0,23.0,41.0,16.0,16.0,BK27
7,MN25,Battery Park City-Lower Manhattan,39699,82.7,21216,39699,39699,1.871182,22.8,39.0,11.1,17.0,1.5,MN25
8,BK31,Bay Ridge,78607,50.5,34991,78607,78607,2.246492,12.3,53.0,28.0,18.0,12.9,BK31
9,QN46,Bayside-Bayside Hills,43808,23.6,16008,43808,43808,2.736632,7.7,50.0,21.6,10.0,5.4,QN46
